In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geohash2

In [10]:
def preprocessing (csv):
    # Read source data from csv file
    df = pd.read_csv(csv)
    
    # Get unique geohash6 location
    geohash = df['geohash6']
    geohash = geohash.drop_duplicates()
    
    # Get min and max day
    max_day = max(df['day'].drop_duplicates())
    min_day = min(df['day'].drop_duplicates())
    
    # Sort dataframe by geohash6, day and timestamp
    d = df.sort_values(by=['geohash6', 'day', 'timestamp'])
    
    # Create a new column by combining day and timestamp
    d['datetime'] = (d['day']).astype(str) + ':' + (d['timestamp']).astype(str)
    
    # Create timestep, each represent 15 minutes
    minute_step = d['datetime'].drop_duplicates().apply(lambda x: pd.Series(minute_since(x), index=['datetime', '15_minute_step']))
   
    # Merge with previous dataframe
    d = pd.merge(d, minute_step, on='datetime', how='inner')
    d.sort_values(by=['15_minute_step'], inplace=True)
    
    # Create a complete dataframe based on number of day and timestep with the demand equal to zero
    # In the previous EDA notebook, we noticed that there're missing data, which actually represent zero demand
    # So we will fill the dataframe with zero demand at the missing timestamp-geohash6 to get a fully sequential dataframe
    complete = []
    upper_bound = max_day * 96
    for geo in geohash:
        for n in range(0, upper_bound):
            complete.append([geo, n])
            
    # Merge the complete dataframe with the previous dataframe
    complete_df = pd.DataFrame(complete, columns=['geohash6', '15_minute_step'])
    final_df = pd.merge(d, complete_df, how='right', on=['geohash6', '15_minute_step'])
    
    # Drop dupplicate rows
    final_df = final_df.drop(columns=['day', 'timestamp', 'datetime'])
    
    # Replace NaN with 0
    final_df = final_df.fillna(0)
    
    # Extract latitude and longitude from geohash6
    coord = geohash.apply(lambda x: pd.Series(decode_geo(x), index=['geohash6','lat', 'lng']))
    coord = coord.drop_duplicates()
    
    # Bin the latitude and logitude to 5 groups
    # Why 5 groups, it's an arbitrar number since the latitude and logitude were anonymized. 
    # We can't really get human insight about the city, blocks and point of interest...
    coord['binned_lat'] = pd.cut(coord['lat'], 5, labels=["lat1", "lat2", "lat3", "lat4", "lat5"])
    coord['binned_lng'] = pd.cut(coord['lng'], 5, labels=["lng1", "lng2", "lng3", "lng4", "lng5"])
    
    # Create new cross feature from binned latitude and binned logitude
    # This will create 25 blocks where each geohash6 will be located in
    # Latitude or Longitude alone will not explain the demand
    # The demand could vary depend on the place, timestamp and day of the week
    # Example1: The demand around shopping mall is huge at evening on weekend
    # Example2: The demand around office is huge at morning or lunch time on week day
    coord['crossed_lat_lng'] = (coord['binned_lat']).astype(str) + '_' + (coord['binned_lng']).astype(str)
    final_df = pd.merge(final_df, coord, on='geohash6', how='inner')
    
    # Create new columns day, hour, minute, day_of_week, and ts
    final_df['day'] = ((final_df['15_minute_step'] * 15) / 1440).astype(np.int8) + 1
    final_df['hour'] = (((final_df['15_minute_step'] * 15) % 1440) / 60).astype(np.int16)
    final_df['minute'] = ((final_df['15_minute_step'] * 15) % 60).astype(np.int16)
    # day_of_week [0,1,2,3,4,5,6] is a sequential order of days in one week
    final_df['day_of_week'] = (final_df['day'] % 7).astype(np.int8)
    final_df['ts'] = final_df['hour'].astype(str) + ":" + final_df['minute'].astype(str)
    final_df.sort_values(by=['15_minute_step', 'geohash6'], inplace=True)
    
    #Downcast
    final_df['demand'] = final_df['demand'].astype(np.float32)
    final_df['15_minute_step'] = final_df['15_minute_step'].astype(np.int16)    
    final_df['geohash6'] = final_df['geohash6'].astype('category')
    final_df['crossed_lat_lng'] = final_df['crossed_lat_lng'].astype('category')
    final_df['ts'] = final_df['ts'].astype('category')
    
    #Drop unused columns
    final_df.drop(columns=['lat', 'lng', 'binned_lat', 'binned_lng', 'hour', 'minute'], inplace=True)
    
    return final_df

In [8]:
# Helper functions
# 1hour = 15mins x 4
# 1day = 4 x 24 = 96
# 61days = 96 *61 = 5856
# In 61 days, we expect 5856 rows for each geohash6

# minute_since: create timestep from datetime starting from 0 to 5855
def minute_since(x):
    arr = x.split(":")
    minutes = ((int(arr[0]) -1 ) * 1440 + int(arr[1]) * 60 + int(arr[2])) / 15
    return x, minutes

# decode_geo: extract the latitude and logitude from geohash6
def decode_geo(geo):
    extracted = geohash2.decode_exactly(geo)[0:2]
    return [geo, extracted[0], extracted[1]]

In [11]:
# Load the data
ds = preprocessing('training.csv')
ds.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7782624 entries, 6207937 to 7243871
Data columns (total 7 columns):
geohash6           category
demand             float32
15_minute_step     int16
crossed_lat_lng    category
day                int8
day_of_week        int8
ts                 category
dtypes: category(3), float32(1), int16(1), int8(2)
memory usage: 148.5 MB


In [12]:
# Order the dataframe
ds.sort_values(by=['geohash6', '15_minute_step', 'day'], inplace=True)
ds.head()

,geohash6,demand,15_minute_step,crossed_lat_lng,day,day_of_week,ts
6207937,qp02yc,0.0,0,lat1_lng1,1,1,0:0
6207938,qp02yc,0.0,1,lat1_lng1,1,1,0:15
6207939,qp02yc,0.0,2,lat1_lng1,1,1,0:30
6207940,qp02yc,0.0,3,lat1_lng1,1,1,0:45
6207941,qp02yc,0.0,4,lat1_lng1,1,1,1:0


In [13]:
# Output the prerprocessed file
ds.to_csv('features.csv', index=False)
# Please proceed with the 3rd notebook: 3_Model.ipynb